from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph()

weather_query = """
LOAD CSV WITH HEADERS FROM 'file:///C:/Users/Admin/Desktop/WeatherRag/Data.csv' AS row
WITH row,
     split(row.Date, "/") AS dateParts
CREATE (w:weatherDate {
    Year: toInteger(row.Year),
    Month: toInteger(row.Month),
    Tmax: toFloat(row.Tmax),
    Tmin: toFloat(row.Tmin),
    AF: toFloat(row.AF),
    Rain: toFloat(row.Rain),
    Sun: toFloat(row.Sun),
    Date: date(dateParts[2] + "-" + dateParts[1] + "-" + dateParts[0]),
    Tmean: toFloat(row.Tmean)
  
})
  RETURN row.Year
    LIMIT 10
"""

graph.query(weather_query)

from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph()

weather_query = """
LOAD CSV WITH HEADERS FROM 'file:///C:/Users/Admin/Desktop/WeatherRag/Data.csv' AS row
WITH row,
   toInteger(coalesce(trim(row.Year), '0')) AS year,
    toInteger(coalesce(trim(row.Month), '0')) AS month,
    toFloat(coalesce(trim(row.Tmax), '0')) AS tmax,
    toFloat(coalesce(trim(row.Tmin), '0')) AS tmin,
    toFloat(coalesce(trim(row.AF), '0')) AS af,
    toFloat(coalesce(trim(row.Rain), '0')) AS rain,
    toFloat(coalesce(trim(row.Sun), '0')) AS sun,
    split(coalesce(trim(row.Date), '01/01/1970'), "/") AS dateParts,
    toFloat(coalesce(trim(row.Tmean), '0')) AS tmean

MERGE (y:Year {year:year})
MERGE (m:Month {month: month, name: CASE month
    WHEN 1 THEN 'January' 
    WHEN 2 THEN 'February' 
    WHEN 3 THEN 'March' 
    WHEN 4 THEN 'April' 
    WHEN 5 THEN 'May' 
    WHEN 6 THEN 'June' 
    WHEN 7 THEN 'July' 
    WHEN 8 THEN 'August' 
    WHEN 9 THEN 'September' 
    WHEN 10 THEN 'October' 
    WHEN 11 THEN 'November' 
    WHEN 12 THEN 'December' 
    END,

    Tmax: tmax,
    Tmin: tmin,
    AF: af,
    Rain: rain,
    Sun: sun,
    Tmean: tmean,
    Date: date(dateParts[2] + "-" + dateParts[1] + "-" + dateParts[0])

    })

MERGE (y)-[:HAS_MONTH]->(m)

"""

graph.query(weather_query)

SEARCH QUERIES

In [35]:
#Get all weather data for a specific year:
search_query = """
MATCH (y:Year {year: 1963})-[:HAS_MONTH]->(m:Month)
RETURN m.name AS Month, m.Tmax AS MaxTemperature, m.Tmin AS MinTemperature, m.Rain AS Rainfall, m.Sun AS Sunlight, m.Tmean AS MeanTemperature
ORDER BY m.month
"""
graph.query(search_query)

[{'Month': 'January',
  'MaxTemperature': -0.1,
  'MinTemperature': -5.7,
  'Rainfall': 24.7,
  'Sunlight': 58.0,
  'MeanTemperature': -2.9},
 {'Month': 'February',
  'MaxTemperature': 1.2,
  'MinTemperature': -3.5,
  'Rainfall': 13.2,
  'Sunlight': 59.0,
  'MeanTemperature': -1.15},
 {'Month': 'March',
  'MaxTemperature': 9.9,
  'MinTemperature': 2.8,
  'Rainfall': 50.3,
  'Sunlight': 102.6,
  'MeanTemperature': 6.35},
 {'Month': 'April',
  'MaxTemperature': 13.4,
  'MinTemperature': 4.7,
  'Rainfall': 41.0,
  'Sunlight': 124.9,
  'MeanTemperature': 9.05},
 {'Month': 'May',
  'MaxTemperature': 15.8,
  'MinTemperature': 6.3,
  'Rainfall': 43.5,
  'Sunlight': 189.5,
  'MeanTemperature': 11.05},
 {'Month': 'June',
  'MaxTemperature': 20.6,
  'MinTemperature': 10.0,
  'Rainfall': 43.0,
  'Sunlight': 201.8,
  'MeanTemperature': 15.3},
 {'Month': 'July',
  'MaxTemperature': 20.6,
  'MinTemperature': 10.6,
  'Rainfall': 52.4,
  'Sunlight': 170.1,
  'MeanTemperature': 15.6},
 {'Month': 'Augus

In [38]:
#maximum rain in a month of specific year
search_query = """
MATCH (y:Year {year: 1963})-[:HAS_MONTH]->(m:Month)
Return m.name as month, m.Rain as Rainfall 
ORDER BY m.Rain DESC
LIMIT 1
"""
graph.query(search_query)

[{'month': 'August', 'Rainfall': 100.4}]

In [39]:
#Calculate the average temperature for a specific year:
search_query = """
MATCH(y:Year {year:1963})-[:HAS_MONTH]->(m:Month) 
return y.year As Year, avg(m.Tmean) AS AverageTemperature
"""
graph.query(search_query)


[{'Year': 1963, 'AverageTemperature': 8.608333333333334}]

In [41]:
#Retrieve all weather data for a specific month across all years:
search_query = """
MATCH(y:Year)-[:HAS_MONTH]->(m:Month {name:'January'})
RETURN y.year AS Year, m.Tmax AS MaxTemperature, m.Tmin AS MinTemperature, m.Rain AS Rainfall, m.Sun AS Sunlight, m.Tmean AS MeanTemperature
ORDER BY y.year
"""
graph.query(search_query)

[{'Year': 1959,
  'MaxTemperature': 4.4,
  'MinTemperature': -1.4,
  'Rainfall': 0.0,
  'Sunlight': 78.1,
  'MeanTemperature': 1.5},
 {'Year': 1960,
  'MaxTemperature': 6.7,
  'MinTemperature': 1.6,
  'Rainfall': 0.0,
  'Sunlight': 36.3,
  'MeanTemperature': 4.15},
 {'Year': 1961,
  'MaxTemperature': 6.2,
  'MinTemperature': 1.0,
  'Rainfall': 55.9,
  'Sunlight': 50.5,
  'MeanTemperature': 3.6},
 {'Year': 1962,
  'MaxTemperature': 7.3,
  'MinTemperature': 0.7,
  'Rainfall': 62.3,
  'Sunlight': 73.5,
  'MeanTemperature': 4.0},
 {'Year': 1963,
  'MaxTemperature': -0.1,
  'MinTemperature': -5.7,
  'Rainfall': 24.7,
  'Sunlight': 58.0,
  'MeanTemperature': -2.9},
 {'Year': 1964,
  'MaxTemperature': 5.0,
  'MinTemperature': -0.3,
  'Rainfall': 18.7,
  'Sunlight': 45.8,
  'MeanTemperature': 2.35},
 {'Year': 1965,
  'MaxTemperature': 6.1,
  'MinTemperature': 0.9,
  'Rainfall': 47.4,
  'Sunlight': 78.3,
  'MeanTemperature': 3.5},
 {'Year': 1966,
  'MaxTemperature': 4.6,
  'MinTemperature': 0.4

In [13]:
#Get the year with the highest average temperature for a specific month:
search_query = """
MATCH (y:Year)-[:HAS_MONTH]->(m:Month {name: 'January'})
RETURN y.year AS year , avg(m.Tmean) as AverageTemperature
ORDER BY AverageTemperature DESC
LIMIT 1
"""
graph.query(search_query)

[{'year': 1989, 'AverageTemperature': 6.6}]

In [16]:


#Average temperature between two given years
search_query = """
MATCH (y:Year)-[:HAS_MONTH]->(m:Month)
WHERE y.year >= 1980 AND y.year <= 1990
RETURN AVG(m.Rain) AS average_rainfall
"""
graph.query(search_query)

[{'average_rainfall': 56.56363636363639}]

In [1]:
from langchain.chains import GraphCypherQAChain
from langchain_groq import ChatGroq
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph()

llm = ChatGroq(
    model = "llama-3.1-70b-versatile",
    temperature = 0,
    max_tokens = 2000
)

chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True)


In [33]:
from langchain_core.prompts import ChatPromptTemplate
system_message = """

You are interacting with a Neo4j graph database containing weather data structured as follows:

Graph Structure:
        Year Node (Year)
        Properties:
        year (Integer): 

        Month Node (Month)
        Properties:
        month (Integer): The numeric representation of the month (1 = January, 2 = February, ..., 12 = December).
        name (String): The name of the month (e.g., "January", "February").
        Tmax (Float): Maximum temperature recorded in the month.
        Tmin (Float): Minimum temperature recorded in the month.
        AF (Float): Number of air frosts in the month.
        Rain (Float): Rainfall recorded in millimeters.
        Sun (Float): Sunlight recorded in hours.
        Tmean (Float): Mean temperature in the month.
        Date (Date): Date representation (e.g., "1963-01-01").

        Relationships:
        HAS_MONTH: Connects Year nodes to their respective Month nodes.

Task:
        You are provided with a natural language query. Your goal is to:
        Understand the user's intent: Identify what the user is asking for, such as retrieving data, finding averages, or comparing values, average between to given dates or months or years
        Generate an appropriate Cypher query: Based on the graph structure and properties provided above, translate the user's natural language query into a Cypher query.
        Return the Cypher query: Ensure the query is correctly formatted and accounts for all relevant properties.

        Use logical operators like >= and <= for filtering node for between specific years, months or dtaes.DO NOT use range() for this
        WHERE clauses should be used for property value comparisons 
        


Examples of Natural Language Queries:
        "What was the average temperature in a given year on month?"
        "Show me the month with the most rainfall in given year."
        "Get all the weather data for January across all years."
        "Which year had the highest average temperature in July?"
        "what was the average temperature between two years or months or dates"
        "what was the maximum temperature between two years or months or dates"

Given a natural language query, generate a Cypher query that accurately reflects the user's request,
ensuring that any specific year mentioned in the query is directly translated into the Cypher query. For instance, 
if the user asks for data from 1991 or in 1991, the Cypher query should explicitly reference the year 1991 in the `MATCH` statement.
if the user asks for data from january or in january or any other month, the Cypher query should explicitly reference the month anuary or month 1 in the `MATCH` statement.
ALways give answer obtained from data and never give any suggestion or interpretation

User question: {query}


"""

prompt = ChatPromptTemplate.from_template(system_message)

In [34]:
from langchain.chains import GraphCypherQAChain
from langchain_groq import ChatGroq
from langchain_community.graphs import Neo4jGraph
graph = Neo4jGraph()


llm = ChatGroq(
    model = "llama-3.1-70b-versatile",
    temperature = 0,
    max_tokens = 2000
)



chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True,  cypher_prompt = prompt)

In [36]:
# giving wrong XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
question = 'what was average rain in between 1980 and 1990'  

response = chain.invoke({"query": question})
response








> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (y:Year {year: >= 1980 AND year: <= 1990})-[:HAS_MONTH]->(m:Month)
RETURN avg(m.Rain) AS average_rainfall



ValueError: Generated Cypher Statement is not valid
{code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input '>=': expected an expression (line 2, column 22 (offset: 28))
"MATCH (y:Year {year: >= 1980 AND year: <= 1990})-[:HAS_MONTH]->(m:Month)"
                      ^}

In [37]:
# giving wrong XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
question = 'what was average temperature in between 1980 and 1990'  

response = chain.invoke({"query": question})
response




> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (y:Year {year: >= 1980 AND year: <= 1990})-[:HAS_MONTH]->(m:Month)
RETURN avg(m.Tmean) AS average_temperature



ValueError: Generated Cypher Statement is not valid
{code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input '>=': expected an expression (line 2, column 22 (offset: 28))
"MATCH (y:Year {year: >= 1980 AND year: <= 1990})-[:HAS_MONTH]->(m:Month)"
                      ^}



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (y:Year {year: range(1980, 1990)})-[:HAS_MONTH]->(m:Month)
RETURN avg(m.Rain) AS average_rainfall

Full Context:
[{'average_rainfall': None}]

> Finished chain.
verbose response is {'query': 'what was average rain in between 1980 and 1990', 'result': "I don't know the answer."}


In [1]:
import io

import re
from contextlib import redirect_stdout

question = 'what was average temperature in december 1985'
if question:
    try:
        f = io.StringIO() # create a place to capture output
        with redirect_stdout(f):  # ouput(response) is redirected and sotored in f 
            response = chain.invoke({"query": question})
        verbose_output = f.getvalue()
        if verbose_output:
            clean_output = re.sub(r'\x1b\[[0-9;]*m', '', verbose_output)
            print(response)
            print(clean_output)
        else:
            print("Could not extract Cypher query.")
    except Exception as e:
        print(f"An error occurred: {e}")
else:
    print("Please enter a question.")


An error occurred: name 'chain' is not defined
